## Balancing immediate and long-term goals

In [49]:
import warnings; warnings.filterwarnings(action='ignore')

import gym
import numpy as np

import random

np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)

In [50]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title="Policy:"):
    print(title)
    arrs = {k:v for k, v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s+1)%n_cols == 0:
            print("|")

In [51]:
def print_state_value_function(V, P, n_cols=4, prec=3, title="State-value function:"):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), f"{np.round(v, prec)}".rjust(6), end=" ")
        if (s+1)%n_cols == 0:
            print("|")

In [52]:
from tabulate import tabulate
def print_action_value_function(Q, optimal_Q=None, action_symbols=('<', '>'), prec=3, title='Action-value function'):
    vf_types=('',) if optimal_Q is None else ('', '*', 'err')
    headers = ['s',] + [' '.join(i) for i in list(itertoosl.product(vf_types, action_symbols))]
    print(title)
    states = np.arange(len(Q))[..., np.newaxis]
    arr = np.hstack((states, np.round(Q, prec)))
    if not (optimal_Q is None):
        arr = np.hstack((arr, np.round(optimal_Q,prec), np.round(optimal_Q - Q, prec)))
    print(tabulate(arr, headers, tablefmt="fancy_grid"))

In [53]:
def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            state, _, done, _ = env.step(pi(state))
            steps += 1
        results.append(state == goal_state)
    return np.sum(results)/len(results)

In [54]:
def mean_return(env, pi, n_episodes=100, max_steps=200):
    env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        results.append(0.)
        while not done and steps < max_steps:
            state, reward, done, _ = env.step(pi(state))
            results[-1]  += reward
            steps += 1
    return np.mean(results)

### Frozen Lake MDP and sample policies

In [55]:
env = gym.make('FrozenLake-v1')
P = env.env.P
init_state = env.reset()
goal_state = 15

LEFT, DOWN, RIGHT, UP = range(4)
random_pi = lambda s: {
    0: RIGHT, 1: LEFT,   2: DOWN,  3: UP,
    4: LEFT,  5: LEFT,   6: RIGHT, 7: LEFT,
    8: UP,    9: DOWN,   10:  UP,  11: LEFT,
    12: LEFT, 13: RIGHT, 14: DOWN, 15: LEFT
}[s]
print_policy(random_pi, P)
print(f"Reaches goal {probability_success(env, random_pi, goal_state=goal_state)*100:.2f}%")
print(f"Obtains an average undiscounted return of {mean_return(env, random_pi):.4f}")

Policy:
| 00      > | 01      < | 02      v | 03      ^ |
| 04      < |           | 06      > |           |
| 08      ^ | 09      v | 10      ^ |           |
|           | 13      > | 14      v |           |
Reaches goal 8.00%
Obtains an average undiscounted return of 0.0800


In [56]:
go_get_pi = lambda s: {
    0: RIGHT, 1: RIGHT,  2: DOWN,   3: LEFT,
    4: DOWN,  5: LEFT,   6: DOWN,   7: LEFT,
    8: RIGHT, 9: RIGHT,  10: DOWN,  11: LEFT,
    12: LEFT, 13: RIGHT, 14: RIGHT, 15: LEFT
}[s]
print_policy(go_get_pi, P)
print(f"Reaches goal {probability_success(env, go_get_pi, goal_state=goal_state)*100:.2f}%")
print(f"Obtains an average undiscounted return of {mean_return(env, go_get_pi):.4f}")

Policy:
| 00      > | 01      > | 02      v | 03      < |
| 04      v |           | 06      v |           |
| 08      > | 09      > | 10      v |           |
|           | 13      > | 14      > |           |
Reaches goal 4.00%
Obtains an average undiscounted return of 0.0400


In [57]:
careful_pi = lambda s: {
    0: LEFT,  1: UP,     2: UP,     3: UP,
    4: LEFT,  5: LEFT,   6: UP,     7: LEFT,
    8: UP,    9: DOWN,   10: LEFT,  11: LEFT,
    12: LEFT, 13: RIGHT, 14: RIGHT, 15: LEFT
}[s]
print_policy(careful_pi, P)
print(f"Reaches goal {probability_success(env, careful_pi, goal_state=goal_state)*100:.2f}%")
print(f"Obtains an average undiscounted return of {mean_return(env, careful_pi):.4f}")

Policy:
| 00      < | 01      ^ | 02      ^ | 03      ^ |
| 04      < |           | 06      ^ |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      > |           |
Reaches goal 56.00%
Obtains an average undiscounted return of 0.5600


### Policy Evaluation

In [58]:
def policy_evaluation(pi, P, gamma=1., theta=1e-10):
    prev_V = np.zeros(len(P), dtype=np.float64)
    while True:
        V = np.zeros(len(P), dtype=np.float64)
        for s in range(len(P)):
            for prob, next_state, reward, done in P[s][pi(s)]:
                V[s] += prob*(reward + gamma*prev_V[next_state]*(not done))
        if np.max(np.abs(prev_V - V)) < theta:
            break
        prev_V = V.copy()
    return V

In [59]:
V = policy_evaluation(careful_pi, P, gamma=0.99)
print_state_value_function(V, P, prec=4)

State-value function:
| 00 0.4079 | 01 0.3754 | 02 0.3543 | 03 0.3438 |
| 04 0.4203 |           | 06 0.1169 |           |
| 08 0.4454 | 09  0.484 | 10 0.4328 |           |
|           | 13 0.5884 | 14 0.7107 |           |


### Pocliy Improvement

In [60]:
def policy_improvement(V, P, gamma=1.):
    Q = np.zeros((len(P), len(P[0])), dtype=np.float64)
    for s in range(len(P)):
        for a in range(len(P[s])):
            for prob, next_state, reward, done in P[s][a]:
                Q[s][a] += prob*(reward + gamma*V[next_state]*(not done))
    new_pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]
    return new_pi

In [61]:
careful_pi_plus = policy_improvement(V, P, gamma=0.99)
print_policy(careful_pi_plus, P)
print(f"Reaches goal {probability_success(env, careful_pi_plus, goal_state=goal_state)*100:.2f}%")
print(f"Obtains an average undiscounted return of {mean_return(env, careful_pi_plus):.4f}")

Policy:
| 00      < | 01      ^ | 02      ^ | 03      ^ |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |
Reaches goal 69.00%
Obtains an average undiscounted return of 0.6900


In [62]:
new_V = policy_evaluation(careful_pi_plus, P, gamma=0.99)
print_state_value_function(new_V, P, prec=4)

State-value function:
| 00  0.542 | 01 0.4988 | 02 0.4707 | 03 0.4569 |
| 04 0.5585 |           | 06 0.3583 |           |
| 08 0.5918 | 09 0.6431 | 10 0.6152 |           |
|           | 13 0.7417 | 14 0.8628 |           |


In [63]:
print_state_value_function(new_V - V, P, prec=4)

State-value function:
| 00 0.1341 | 01 0.1234 | 02 0.1164 | 03  0.113 |
| 04 0.1381 |           | 06 0.2414 |           |
| 08 0.1464 | 09 0.1591 | 10 0.1824 |           |
|           | 13 0.1533 | 14 0.1521 |           |


### Alternating between evaluation and improvement

In [64]:
adversarial_pi = lambda s: {
    0: UP,  1: UP,     2: UP,     3: UP,
    4: UP,  5: LEFT,   6: UP,     7: LEFT,
    8: LEFT,    9: LEFT,   10: LEFT,  11: LEFT,
    12: LEFT, 13: LEFT, 14: LEFT, 15: LEFT
}[s]
print_policy(adversarial_pi, P)
print(f"Reaches goal {probability_success(env, adversarial_pi, goal_state=goal_state)*100:.2f}%")
print(f"Obtains an average undiscounted return of {mean_return(env, adversarial_pi):.4f}")

Policy:
| 00      ^ | 01      ^ | 02      ^ | 03      ^ |
| 04      ^ |           | 06      ^ |           |
| 08      < | 09      < | 10      < |           |
|           | 13      < | 14      < |           |
Reaches goal 0.00%
Obtains an average undiscounted return of 0.0000


In [65]:
V = policy_evaluation(adversarial_pi, P, gamma=0.99)
print_state_value_function(V, P, prec=2)

State-value function:
| 00    0.0 | 01    0.0 | 02    0.0 | 03    0.0 |
| 04    0.0 |           | 06    0.0 |           |
| 08    0.0 | 09    0.0 | 10    0.0 |           |
|           | 13    0.0 | 14    0.0 |           |


In [66]:
i_pi = policy_improvement(V, P, gamma=0.99)
print_policy(i_pi, P)
print(f"Reaches goal {probability_success(env, i_pi, goal_state=goal_state)*100:.2f}%")
print(f"Obtains an average undiscounted return of {mean_return(env, i_pi):.4f}")

Policy:
| 00      < | 01      < | 02      < | 03      < |
| 04      < |           | 06      < |           |
| 08      < | 09      < | 10      < |           |
|           | 13      < | 14      v |           |
Reaches goal 0.00%
Obtains an average undiscounted return of 0.0000


In [67]:
i_V = policy_evaluation(i_pi, P, gamma=0.99)
print_state_value_function(i_V, P, prec=2)

State-value function:
| 00    0.0 | 01    0.0 | 02   0.04 | 03   0.02 |
| 04    0.0 |           | 06   0.07 |           |
| 08    0.0 | 09    0.0 | 10   0.19 |           |
|           | 13    0.0 | 14    0.5 |           |


In [68]:
ii_pi = policy_improvement(i_V, P, gamma=0.99)
print_policy(ii_pi, P)
print(f"Reaches goal {probability_success(env, ii_pi, goal_state=goal_state)*100:.2f}%")
print(f"Obtains an average undiscounted return of {mean_return(env, ii_pi):.4f}")

Policy:
| 00      < | 01      v | 02      > | 03      ^ |
| 04      < |           | 06      < |           |
| 08      < | 09      v | 10      < |           |
|           | 13      v | 14      > |           |
Reaches goal 0.00%
Obtains an average undiscounted return of 0.0000


In [69]:
ii_V = policy_evaluation(ii_pi, P, gamma=0.99)
print_state_value_function(ii_V, P, prec=2)

State-value function:
| 00    0.0 | 01   0.05 | 02   0.16 | 03   0.15 |
| 04    0.0 |           | 06   0.17 |           |
| 08    0.0 | 09   0.22 | 10   0.35 |           |
|           | 13   0.33 | 14   0.67 |           |


In [70]:
iii_pi = policy_improvement(ii_V, P, gamma=0.99)
print_policy(iii_pi, P)
print(f"Reaches goal {probability_success(env, iii_pi, goal_state=goal_state)*100:.2f}%")
print(f"Obtains an average undiscounted return of {mean_return(env, iii_pi):.4f}")

Policy:
| 00      v | 01      > | 02      > | 03      ^ |
| 04      < |           | 06      < |           |
| 08      v | 09      v | 10      < |           |
|           | 13      > | 14      > |           |
Reaches goal 19.00%
Obtains an average undiscounted return of 0.1900


In [71]:
iii_V = policy_evaluation(iii_pi, P, gamma=0.99)
print_state_value_function(iii_V, P, prec=2)

State-value function:
| 00   0.12 | 01   0.09 | 02   0.19 | 03   0.19 |
| 04   0.15 |           | 06    0.2 |           |
| 08   0.19 | 09   0.38 | 10   0.43 |           |
|           | 13   0.53 | 14   0.71 |           |


In [72]:
iiii_pi = policy_improvement(iii_V, P, gamma=0.99)
print_policy(iiii_pi, P)
print(f"Reaches goal {probability_success(env, iiii_pi, goal_state=goal_state)*100:.2f}%")
print(f"Obtains an average undiscounted return of {mean_return(env, iiii_pi):.4f}")

Policy:
| 00      < | 01      ^ | 02      > | 03      ^ |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |
Reaches goal 66.00%
Obtains an average undiscounted return of 0.6600


In [73]:
iiii_V = policy_evaluation(iiii_pi, P, gamma=0.99)
print_state_value_function(iiii_V, P, prec=2)

State-value function:
| 00   0.52 | 01   0.38 | 02   0.26 | 03   0.25 |
| 04   0.54 |           | 06   0.28 |           |
| 08   0.57 | 09   0.62 | 10   0.58 |           |
|           | 13   0.72 | 14   0.85 |           |


### Policy Iteration

In [74]:
def policy_iteration(P, gamma=1., theta=1e-10):
    random_actions = np.random.choice(tuple(P[0].keys()), len(P))
    pi = lambda s: {s:a for s, a in enumerate(random_actions)}[s]
    while True:
        old_pi = {s:pi(s) for s in range(len(P))}
        V = policy_evaluation(pi, P, gamma, theta)
        pi = policy_improvement(V, P, gamma)
        if old_pi == {s:pi(s) for s in range(len(P))}:
            break
    return V, pi

In [75]:
V_best_p, pi_best_p = policy_iteration(P, gamma=0.99)
print("Optimal policy and state-value function (PI):")
print_policy(pi_best_p, P)
print(f"Reaches goal {probability_success(env, pi_best_p, goal_state=goal_state)*100:.2f}%")
print(f"Obtains an average undiscounted return of {mean_return(env, pi_best_p):.4f}")
print()
print_state_value_function(V_best_p, P, prec=4)

Optimal policy and state-value function (PI):
Policy:
| 00      < | 01      ^ | 02      ^ | 03      ^ |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |
Reaches goal 69.00%
Obtains an average undiscounted return of 0.6900

State-value function:
| 00  0.542 | 01 0.4988 | 02 0.4707 | 03 0.4569 |
| 04 0.5585 |           | 06 0.3583 |           |
| 08 0.5918 | 09 0.6431 | 10 0.6152 |           |
|           | 13 0.7417 | 14 0.8628 |           |


### Value Iteration

In [76]:
def value_iteration(P, gamma=1., theta=1e-10):
    V = np.zeros(len(P), dtype=np.float64)
    while True:
        Q = np.zeros((len(P), len(P[0])), dtype=np.float64)
        for s in range(len(P)):
            for a in range(len(P[s])):
                for prob, next_state, reward, done in P[s][a]:
                    Q[s][a] += prob*(reward+gamma*V[next_state]*(not done))
        if np.max(np.abs(V - np.max(Q, axis=1))) < theta:
            break
        V = np.max(Q, axis=1)
    pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]
    return V, pi

In [77]:
V_best_v, pi_best_v = value_iteration(P, gamma=0.99)
print("Optimal policy and state-value function (VI)")
print_policy(pi_best_v, P)
print(f"Reaches goal {probability_success(env, pi_best_v, goal_state=goal_state)*100:.2f}%")
print(f"Obtains an average undiscounted return of {mean_return(env, pi_best_v):.4f}")
print()
print_state_value_function(V_best_v, P, prec=4)

Optimal policy and state-value function (VI)
Policy:
| 00      < | 01      ^ | 02      ^ | 03      ^ |
| 04      < |           | 06      < |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |
Reaches goal 69.00%
Obtains an average undiscounted return of 0.6900

State-value function:
| 00  0.542 | 01 0.4988 | 02 0.4707 | 03 0.4569 |
| 04 0.5585 |           | 06 0.3583 |           |
| 08 0.5918 | 09 0.6431 | 10 0.6152 |           |
|           | 13 0.7417 | 14 0.8628 |           |


### Russel & Norvig's Gridworld